In [4]:
# !pip install cryptography
# !pip install PyMySQL

You should consider upgrading via the 'G:\Мой диск\Учёба\8 семестр\диплом\Проект\VKR\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.config['SQLALCHEMY_DATABASE_URI'] = 'mysql+pymysql://{username}:{password}@{host}:{port}/{db}?charset=utf8'.format(username="kokon",
            password="TrOlOlO123))", host="127.0.0.1", port=3306, db="prediction")
db = SQLAlchemy(app)


In [2]:
class Subject(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class Results(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    succes_done = db.Column(db.Boolean, default=False)

class Comands(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

class Scores(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    subject_id = db.Column(db.Integer, db.ForeignKey('subject.id'), nullable=False)
    subject = db.relationship('Subject', backref=db.backref('scores', lazy=True))
    result_id = db.Column(db.Integer, db.ForeignKey('results.id'), nullable=False)
    result = db.relationship('Results', backref=db.backref('scores', lazy=True))
    comands_id = db.Column(db.Integer, db.ForeignKey('comands.id'), nullable=False)
    comands = db.relationship('Comands', backref=db.backref('scores', lazy=True))
    visiting = db.Column(db.Float, default=0.0)
    scores = db.Column(db.Float, default=0.0)
    test_number = db.Column(db.Integer, nullable=False)

In [13]:
import tkinter.filedialog as tkFileDialog
def open_file():
    return tkFileDialog.askopenfilename( title='Выберите файл:')
import pandas as pd

data = pd.read_csv(open_file(), delimiter=";", encoding='cp1251', decimal=",") #delimiter=";", 
data = data.dropna()
data

,ФИО,Пол,Команда,Сдал(-а),Посещение до Контрольная работа № 1,Баллы до Контрольная работа № 1,Контрольная работа № 1,Посещение до Контрольная работа № 2,Баллы до Контрольная работа № 2,Контрольная работа № 2
0,Абатуров Георгий Сергеевич,1.0,МА Л-08,1,0.0,0.0,0,0.0,0.0,38
1,Абдимиталипов Мустафа,1.0,МА Л-04,0,0.86,25.0,0,0.73,23.0,20
2,Абдимуминов Абдужалол Абдихаким угли,1.0,МА Л-05,0,0.0,0.0,0,0.0,0.0,0
3,Абдразакова Екатерина Наилевна,0.0,МА Л-04,1,0.79,24.0,0,1.0,22.0,25
4,Абдрашитова Диана Дамировна,0.0,МА Л-07,1,0.93,0.0,0,0.55,20.0,35
...,...,...,...,...,...,...,...,...,...,...
948,Ярмоц Надежда Андреевна,0.0,МА Л-08,1,1.0,5.5,0,0.91,24.5,30
949,Ярославцев Дмитрий Сергеевич,1.0,МА Л-04,0,0.79,2.0,0,0.82,1.0,0
950,Яруллин Тимофей Русланович,1.0,МА Л-08,1,1.0,2.0,0,1.0,19.0,25
951,Ятло Николай Олегович,1.0,МА Л-04,0,0.93,0.0,0,0.55,7.0,0


In [4]:
with app.app_context() as ctx:
    ctx.push()
    db.create_all()

In [14]:
subject_temp = Subject(name="Математический анализ")
with app.app_context():
    db.session.add_all([subject_temp])
    db.session.commit()

In [15]:
exams = [name  for name in data.columns.to_list() if name.startswith("Контрольная")]
with app.app_context():
    for index, d in data.iterrows():
        comand = db.session.query(Comands).filter_by(name=d["Команда"] ).first()
        if not bool(comand):
            comand = Comands(name=d["Команда"])
            db.session.add_all([comand])
            db.session.commit()

        result = Results(succes_done= d["Сдал(-а)"]==1.0)
        db.session.add_all([result])
        db.session.commit()
        print(f'result: {d["Сдал(-а)"]}')
        for ex in exams:
            score = Scores(subject=subject_temp, result=result, comands=comand, visiting=d[f"Посещение до {ex}"], test_number=exams.index(ex)+1)
            if (f"Баллы до {ex}" in data.columns):
                score.scores = d[f"Баллы до {ex}"]
            db.session.add_all([score])
            db.session.commit()

result: 1
result: 0
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 1
result: 1
result: 0
result: 1
result: 0
result: 1
result: 0
result: 1
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 0
result: 0
result: 1
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 1
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 0
result: 1
result: 0
result: 1
result: 0
result: 1
result: 1
result: 1
result: 0
result: 0
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 1
result: 1
result: 0
result: 0
result: 1
result: 0
result: 1
result: 0
result: 0
result: 1
result: 0
result: 1
result: 1
result: 0
result: 1
result: 1
result: 0
result: 1
result: 0
result: 1
result: 1
result: 1
result: 1
result: 0
result: 1
result: 1
result: 0
result: 0
result: 1
result: 1
result: 1
result: 0
result: 0
result: 1
result: 0
result: 1
result: 1
result: 0
result: 1
result: 1
result: 0
result: 0
result: 0
result: 0
result: 1
result: 1
